### upload all csv

In [2]:
from google.colab import files
uploaded = files.upload()

Saving covid-statistics-by-us-states-daily-updates3.csv to covid-statistics-by-us-states-daily-updates3.csv


### Combine into one csv


In [7]:
import pandas as pd
filenamePattern = "covid-statistics-by-us-states-daily-updates"
combined_csv = pd.concat([pd.read_csv(f"{filenamePattern}{i}.csv") for i in range(1, 4) ])
combined_csv.pop('hash')
combined_csv.drop(combined_csv.filter(regex="Unname"),axis=1, inplace=True)
combined_csv.drop(["commercialscore", "grade", "negativeregularscore", "negativescore", "positivescore", "score"],axis=1, inplace=True)
combined_csv = combined_csv.groupby(['state', 'date']).sum().groupby(level=0).cumsum().reset_index()
combined_csv.to_csv( "covid-combination.csv", index=False, encoding='utf-8-sig')
combined_csv

,state,date,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,death,hospitalized,totaltestsviral,positivetestsviral,negativetestsviral,positivecasesviral,deathconfirmed,deathprobable,fips,positiveincrease,negativeincrease,total,totaltestresults,totaltestresultsincrease,posneg,deathincrease,hospitalizedincrease
0,AK,2020-03-06,0.0,24.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,6,0.0,0.0,27.0,24.0,0.0,24.0,0.0,0.0
1,AK,2020-03-07,0.0,60.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,12,0.0,12.0,69.0,60.0,12.0,60.0,0.0,0.0
2,AK,2020-03-08,0.0,102.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,18,0.0,18.0,129.0,102.0,18.0,102.0,0.0,0.0
3,AK,2020-03-09,0.0,171.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,24,0.0,45.0,225.0,171.0,45.0,171.0,0.0,0.0
4,AK,2020-03-10,0.0,240.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,30,0.0,45.0,321.0,240.0,45.0,240.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8188,WY,2020-07-25,144968.0,3139841.0,0.0,1745.0,13546.0,0.0,0.0,0.0,0.0,102364.0,1787.0,13546.0,2956648.0,86020.0,2728817.0,91594.0,0.0,0.0,15064,3622.0,72102.0,3284809.0,3284809.0,75724.0,3284809.0,43.0,288.0
8189,WY,2020-07-26,147443.0,3185664.0,0.0,1759.0,13704.0,0.0,0.0,0.0,0.0,104247.0,1812.0,13704.0,3024348.0,88051.0,2794486.0,93623.0,0.0,0.0,15120,3651.0,72102.0,3333107.0,3333107.0,75753.0,3333107.0,43.0,288.0
8190,WY,2020-07-27,149963.0,3231487.0,0.0,1776.0,13862.0,0.0,0.0,0.0,0.0,106162.0,1837.0,13862.0,3092048.0,90082.0,2860155.0,95695.0,0.0,0.0,15176,3696.0,72102.0,3381450.0,3381450.0,75798.0,3381450.0,43.0,288.0
8191,WY,2020-07-28,152552.0,3279678.0,0.0,1791.0,14024.0,0.0,0.0,0.0,0.0,108132.0,1863.0,14024.0,3163914.0,92237.0,2929865.0,97831.0,0.0,0.0,15232,3765.0,74470.0,3432230.0,3432230.0,78235.0,3432230.0,44.0,292.0


### Show max and min positive test results

In [9]:
df = pd.read_csv("covid-combination.csv")
dfGroupedMaximus = df.groupby("state").max().reset_index()
positive = dfGroupedMaximus['positive']
max = (dfGroupedMaximus[dfGroupedMaximus.positive == dfGroupedMaximus.positive.max()])
print(f"max: {max.iloc[0].state}, {max.iloc[0].positive}")
min = (dfGroupedMaximus[dfGroupedMaximus.positive == dfGroupedMaximus.positive.min()])
print(f"min: {min.iloc[0].state}, {min.iloc[0].positive}")

max: NY, 60067324.0
min: AS, 0.0


### Show count, mean, std, median of the column “positive”

In [11]:
df = pd.read_csv("covid-combination.csv")
df = df.groupby("state").max()
positive = df['positive']
description = positive.describe()
print(f'count:   {description.count()}')
print(f'mean:    {description.mean()}')
print(f'median:  {description.median()}')
print(f'std:     {description.std()}')

count:   8
mean:    10419295.49261326
median:  3747462.8660714286
std:     20327723.275623124


### Show observation interval of the statistics in days

In [2]:
from datetime import datetime
import pandas as pd
df = pd.read_csv("covid-combination.csv")
startTime = datetime.strptime(min(df['date']), "%Y-%m-%d").date()
endTime = datetime.strptime(max(df['date']), "%Y-%m-%d").date()
print(f"distance: {(endTime - startTime).days}")


distance: 189


### Show the list of the ‘healthiest’ states

In [4]:
df = pd.read_csv("covid-combination.csv")
dfGroupedMaximus = df.groupby("state").max().reset_index()
dfGroupedMaximus = dfGroupedMaximus[dfGroupedMaximus['total'] != 0.0]
dfGroupedMaximus = dfGroupedMaximus.iloc[(dfGroupedMaximus.positive / dfGroupedMaximus.total).sort_values().index] 
print(dfGroupedMaximus.head(10))

   state        date  ...  deathincrease  hospitalizedincrease
3     AS  2020-07-29  ...            0.0                   0.0
27    MP  2020-07-29  ...            6.0                   4.0
0     AK  2020-07-29  ...           41.0                 120.0
13    HI  2020-07-29  ...           59.0                 317.0
29    MT  2020-07-29  ...           85.0                 337.0
54    WV  2020-07-29  ...          218.0                 494.0
51    VT  2020-07-29  ...          157.0                 150.0
12    GU  2020-07-29  ...           15.0                   0.0
31    ND  2020-07-29  ...          160.0                 572.0
35    NM  2020-07-29  ...         1006.0                4234.0

[10 rows x 29 columns]


### Add fake row

In [8]:
from random import *
from datetime import date
import numpy as np
df = pd.read_csv("covid-combination.csv")
randStateName = chr(randint(ord('A'), ord('Z'))) + chr(randint(ord('A'), ord('Z')))
dfGroupedMaximus = df.groupby("state").max().reset_index()
toAppend = {'state' : randStateName,
                'date' :  date.today().strftime("%Y-%m-%d")}
for columnName in dfGroupedMaximus.columns:
    if columnName == 'state' or columnName == 'date': continue

    maxValue = eval(f"dfGroupedMaximus[dfGroupedMaximus.{columnName} == dfGroupedMaximus.{columnName}.max()].iloc[0].{columnName}")
    minValue = eval(f"dfGroupedMaximus[dfGroupedMaximus.{columnName} == dfGroupedMaximus.{columnName}.min()].iloc[0].{columnName}")
    randValue = 0
    if isinstance(minValue, (np.int64)) or isinstance(maxValue, (np.int64)):
        randValue = randint(minValue, maxValue)
    elif isinstance(minValue, (float)) or isinstance(maxValue, (float)):
        randValue = np.random.uniform(minValue, maxValue)
    else:
        exit(-1)
    
    exec("toAppend.update({" + f'"{columnName}": {randValue}' + " })")
df = df.append(toAppend , 
                ignore_index=True)

# df.to_csv("covid-combination.csv", index=False, encoding='utf-8-sig')
df

,state,date,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,death,hospitalized,totaltestsviral,positivetestsviral,negativetestsviral,positivecasesviral,deathconfirmed,deathprobable,fips,positiveincrease,negativeincrease,total,totaltestresults,totaltestresultsincrease,posneg,deathincrease,hospitalizedincrease
0,AK,2020-03-06,0.000000e+00,2.400000e+01,3.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,6,0.000000,0.000000e+00,2.700000e+01,2.400000e+01,0.000000,2.400000e+01,0.000000,0.000000
1,AK,2020-03-07,0.000000e+00,6.000000e+01,9.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,12,0.000000,1.200000e+01,6.900000e+01,6.000000e+01,12.000000,6.000000e+01,0.000000,0.000000
2,AK,2020-03-08,0.000000e+00,1.020000e+02,2.700000e+01,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,18,0.000000,1.800000e+01,1.290000e+02,1.020000e+02,18.000000,1.020000e+02,0.000000,0.000000
3,AK,2020-03-09,0.000000e+00,1.710000e+02,5.400000e+01,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,5.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,24,0.000000,4.500000e+01,2.250000e+02,1.710000e+02,45.000000,1.710000e+02,0.000000,0.000000
4,AK,2020-03-10,0.000000e+00,2.400000e+02,8.100000e+01,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,30,0.000000,4.500000e+01,3.210000e+02,2.400000e+02,45.000000,2.400000e+02,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8189,WY,2020-07-26,1.474430e+05,3.185664e+06,0.000000e+00,1759.000000,1.370400e+04,0.000000,0.000000,0.000000,0.000000,1.042470e+05,1.812000e+03,1.370400e+04,3.024348e+06,8.805100e+04,2.794486e+06,9.362300e+04,0.000000,0.000000,15120,3651.000000,7.210200e+04,3.333107e+06,3.333107e+06,75753.000000,3.333107e+06,43.000000,288.000000
8190,WY,2020-07-27,1.499630e+05,3.231487e+06,0.000000e+00,1776.000000,1.386200e+04,0.000000,0.000000,0.000000,0.000000,1.061620e+05,1.837000e+03,1.386200e+04,3.092048e+06,9.008200e+04,2.860155e+06,9.569500e+04,0.000000,0.000000,15176,3696.000000,7.210200e+04,3.381450e+06,3.381450e+06,75798.000000,3.381450e+06,43.000000,288.000000
8191,WY,2020-07-28,1.525520e+05,3.279678e+06,0.000000e+00,1791.000000,1.402400e+04,0.000000,0.000000,0.000000,0.000000,1.081320e+05,1.863000e+03,1.402400e+04,3.163914e+06,9.223700e+04,2.929865e+06,9.783100e+04,0.000000,0.000000,15232,3765.000000,7.447000e+04,3.432230e+06,3.432230e+06,78235.000000,3.432230e+06,44.000000,292.000000
8192,WY,2020-07-29,1.551800e+05,3.329062e+06,0.000000e+00,1807.000000,1.418900e+04,0.000000,0.000000,0.000000,0.000000,1.101540e+05,1.889000e+03,1.418900e+04,3.237638e+06,9.452200e+04,3.001303e+06,1.000030e+05,0.000000,0.000000,15288,3804.000000,7.566300e+04,3.484242e+06,3.484242e+06,79467.000000,3.484242e+06,44.000000,295.000000
